In [1]:
import warnings
warnings.filterwarnings("ignore")

from datetime import datetime, timedelta
from pandas import DataFrame, concat, date_range, ExcelWriter, to_datetime
import os
from numpy import isnan, inf, mean
import time
import json
from Calculator import Calculator as Calc

from utils import getSchema, changedType
from BacktestBaseFunc import *

parent = os.path.dirname(os.path.abspath("__file__"))
output_path = os.path.join(parent, "Output", "Bull_Trend_With_MACD_Cross")
if not os.path.isdir(output_path):
    os.makedirs(output_path)

c:\users\kevin\appdata\local\programs\python\python37\lib\site-packages\mpl_finance.py:22: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  category=DeprecationWarning)


In [2]:
def Bull_Trend_With_MACD_Cross(df:DataFrame, **kwargs):
    entry_date = None
    exit_date = None
    entry_price = 0
    exit_price = 0
    max_price = 0
    min_price = inf
    sig = pos = 0
#     window = int(kwargs.get('window', 14))
    take_profit = float(kwargs.get('take_profit', .1))
    stop_loss = float(kwargs.get('stop_loss', .1))
    
#     df = Calc.MACD(df.fillna(method='ffill'))
    df = Calc.MA(df.fillna(method='ffill'),[5, 10, 20, 60])
    df = Calc.MACD(df.fillna(method='ffill'))
#     df = Calc.RSI(df.fillna(method='ffill'), [window])
    result = []
    for i, row in enumerate(df.itertuples()):
        if not i: continue
        if isnan(getattr(row,f"MA{60}")): continue
        # Check Signal without pos
        if not sig and not pos:
#             rsi = getattr(row,f"RSI{window}")
            # 多單條件
            long_trend_condi = row.MA5 > row.MA10 and row.MA10 > row.MA20 and row.MA20 > row.MA60
            g_condi = row.DIF >= row.MACD and all(df['DIF'].loc[row.Index-5:row.Index-1] < df['MACD'].loc[row.Index-5:row.Index-1])
            # 空單條件
#             short_trend_condi = row.MA5 < row.MA10 and row.MA10 < row.MA20 and row.MA20 < row.MA60
#             d_condi = row.DIF <= row.MACD and all(df['DIF'].loc[row.Index-5:row.Index-1] > df['MACD'].loc[row.Index-5:row.Index-1])
#             d_condi = row.MA10 < row.EMA10
#             short_rsi = rsi < 50 and rsi > 30
#             green_bar = row.Close < row.Open and row.High < min(row.MA10, row.EMA10)
            if g_condi and long_trend_condi:
                sig = 1
#             elif d_condi and short_trend_condi:
#                 sig = -1
#             if sig:
#                 sl = row.MA10
#                 tp = round(abs(row.Close - row.MA10) * 1.5, 2) * sig + row.Close
            pos, sig = sig, 0
            max_price = min_price = entry_price = row.Close
            entry_date = row.Date
        # Check Signal with pos
        elif not sig and pos:
            max_price = max(row.High, entry_price)
            min_price = min(row.Low, entry_price)
#             print(f"stoploss : {round(sl, 2)}, takeprofit : {round(tp, 2)}")
#             print(row.Open, row.High, row.Low, row.Close, pos)
            
            if pos > 0:
                if row.High >= entry_price * (1+take_profit):
                    exit_price = entry_price * (1+take_profit)
                elif row.Low <= entry_price * (1-stop_loss):
                    exit_price = entry_price * (1-stop_loss)
            else:
                if row.High >= entry_price * (1+stop_loss):
                    exit_price = entry_price * (1+stop_loss)
                elif row.Low <= entry_price * (1-take_profit):
                    exit_price = entry_price * (1-take_profit)
#                 if row.Low <= tp:
#                     exit_price = tp
#                 elif row.High >= sl:
#                     exit_price = sl
            if exit_price:
                exit_price = round(exit_price, 2)
                sig = -pos
        # Exit Market
#         elif sig and pos:
            if sig:
                if not row.Volume or isnan(row.Close): continue
    #             if row.Ticker == '1538':print(row)
    #             exit_price = row.Open
                exit_date = row.Date
#                 print(entry_date, exit_date, entry_price, exit_price, max_price, min_price, pos)
                res = CreateTradeLog(entry_date, exit_date, entry_price, exit_price, max_price, min_price, pos)
#                 print(res)
                result.append(res)
                entry_date = None
                exit_date = None
                entry_price = 0
                exit_price = 0
                max_price = 0
                min_price = inf
                sig = pos = 0
                tp = sl = 0
        if i == df.shape[0]-1 and pos:
            exit_price = row.Close
            if not row.Volume or isnan(row.Open):
                exit_price = df.fillna(method='ffill').Close.iloc[-1]
            
            exit_date = row.Date
            res = CreateTradeLog(entry_date, exit_date, entry_price, exit_price, max_price, min_price, pos)
            result.append(res)
            entry_date = None
            exit_date = None
            entry_price = 0
            exit_price = 0
            max_price = 0
            min_price = inf
            sig = pos = 0
            tp = sl = 0
#     print()
            
    return result

In [3]:
bt_list = StockList()

In [4]:
results = {}
for ticker_info in bt_list:
    print(f"============= Backtest {ticker_info['Ticker']}=============")
    results[ticker_info['Ticker']] = Backtest(Bull_Trend_With_MACD_Cross, ticker_info['Ticker'], bt_period = 20)
#     break

============= Backtest 1101=============
============= Backtest 1102=============
============= Backtest 1103=============
============= Backtest 1104=============
============= Backtest 1108=============
============= Backtest 1109=============
============= Backtest 1110=============
============= Backtest 1201=============
============= Backtest 1203=============
============= Backtest 1210=============
============= Backtest 1213=============
============= Backtest 1215=============
============= Backtest 1216=============
============= Backtest 1217=============
============= Backtest 1218=============
============= Backtest 1219=============
============= Backtest 1220=============
============= Backtest 1225=============
============= Backtest 1227=============
============= Backtest 1229=============
============= Backtest 1231=============
============= Backtest 1232=============
============= Backtest 1233=============
============= Backtest 1234=============
============= Ba

============= Backtest 1909=============
============= Backtest 2002=============
============= Backtest 2006=============
============= Backtest 2007=============
============= Backtest 2008=============
============= Backtest 2009=============
============= Backtest 2010=============
============= Backtest 2012=============
============= Backtest 2013=============
============= Backtest 2014=============
============= Backtest 2015=============
============= Backtest 2017=============
============= Backtest 2020=============
============= Backtest 2022=============
============= Backtest 2023=============
============= Backtest 2024=============
============= Backtest 2025=============
============= Backtest 2027=============
============= Backtest 2028=============
============= Backtest 2029=============
============= Backtest 2030=============
============= Backtest 2031=============
============= Backtest 2032=============
============= Backtest 2033=============
============= Ba

============= Backtest 2516=============
============= Backtest 2520=============
============= Backtest 2524=============
============= Backtest 2527=============
============= Backtest 2528=============
============= Backtest 2530=============
============= Backtest 2534=============
============= Backtest 2535=============
============= Backtest 2536=============
============= Backtest 2537=============
============= Backtest 2538=============
============= Backtest 2539=============
============= Backtest 2540=============
============= Backtest 2542=============
============= Backtest 2543=============
============= Backtest 2545=============
============= Backtest 2546=============
============= Backtest 2547=============
============= Backtest 2548=============
============= Backtest 2597=============
============= Backtest 2601=============
============= Backtest 2603=============
============= Backtest 2605=============
============= Backtest 2606=============
============= Ba

============= Backtest 3550=============
============= Backtest 3557=============
============= Backtest 3563=============
============= Backtest 3576=============
============= Backtest 3583=============
============= Backtest 3588=============
============= Backtest 3591=============
============= Backtest 3593=============
============= Backtest 3596=============
============= Backtest 3605=============
============= Backtest 3607=============
============= Backtest 3617=============
============= Backtest 3622=============
============= Backtest 3645=============
============= Backtest 3653=============
============= Backtest 3661=============
============= Backtest 3665=============
============= Backtest 3669=============
============= Backtest 3673=============
============= Backtest 3679=============
============= Backtest 3682=============
============= Backtest 3686=============
============= Backtest 3694=============
============= Backtest 3701=============
============= Ba

============= Backtest 6443=============
============= Backtest 6449=============
============= Backtest 6451=============
============= Backtest 6456=============
============= Backtest 6464=============
============= Backtest 6477=============
============= Backtest 6491=============
============= Backtest 6504=============
============= Backtest 6505=============
============= Backtest 6515=============
============= Backtest 6525=============
============= Backtest 6531=============
============= Backtest 6533=============
============= Backtest 6541=============
============= Backtest 6552=============
============= Backtest 6558=============
============= Backtest 6573=============
============= Backtest 6579=============
============= Backtest 6581=============
============= Backtest 6582=============
============= Backtest 6591=============
============= Backtest 6592=============
============= Backtest 6598=============
============= Backtest 6605=============
============= Ba

============= Backtest 2743=============
============= Backtest 2745=============
============= Backtest 2752=============
============= Backtest 2754=============
============= Backtest 2755=============
============= Backtest 2916=============
============= Backtest 2924=============
============= Backtest 2926=============
============= Backtest 2937=============
============= Backtest 3064=============
============= Backtest 3066=============
============= Backtest 3067=============
============= Backtest 3071=============
============= Backtest 3073=============
============= Backtest 3078=============
============= Backtest 3081=============
============= Backtest 3083=============
============= Backtest 3085=============
============= Backtest 3086=============
============= Backtest 3088=============
============= Backtest 3089=============
============= Backtest 3092=============
============= Backtest 3093=============
============= Backtest 3095=============
============= Ba

============= Backtest 4160=============
============= Backtest 4161=============
============= Backtest 4162=============
============= Backtest 4163=============
============= Backtest 4167=============
============= Backtest 4168=============
============= Backtest 4171=============
============= Backtest 4173=============
============= Backtest 4174=============
============= Backtest 4175=============
============= Backtest 4183=============
============= Backtest 4188=============
============= Backtest 4192=============
============= Backtest 4198=============
============= Backtest 4205=============
============= Backtest 4207=============
============= Backtest 4303=============
============= Backtest 4304=============
============= Backtest 4305=============
============= Backtest 4401=============
============= Backtest 4402=============
============= Backtest 4406=============
============= Backtest 4413=============
============= Backtest 4416=============
============= Ba

============= Backtest 5603=============
============= Backtest 5604=============
============= Backtest 5609=============
============= Backtest 5701=============
============= Backtest 5703=============
============= Backtest 5704=============
============= Backtest 5820=============
============= Backtest 5864=============
============= Backtest 5878=============
============= Backtest 5902=============
============= Backtest 5903=============
============= Backtest 5904=============
============= Backtest 5905=============
============= Backtest 6015=============
============= Backtest 6016=============
============= Backtest 6020=============
============= Backtest 6021=============
============= Backtest 6023=============
============= Backtest 6026=============
============= Backtest 6101=============
============= Backtest 6103=============
============= Backtest 6104=============
============= Backtest 6109=============
============= Backtest 6111=============
============= Ba

============= Backtest 6712=============
============= Backtest 6716=============
============= Backtest 6727=============
============= Backtest 6728=============
============= Backtest 6732=============
============= Backtest 6733=============
============= Backtest 6747=============
============= Backtest 6751=============
============= Backtest 6752=============
============= Backtest 6762=============
============= Backtest 6767=============
============= Backtest 6803=============
============= Backtest 7402=============
============= Backtest 7556=============
============= Backtest 8024=============
============= Backtest 8027=============
============= Backtest 8032=============
============= Backtest 8034=============
============= Backtest 8038=============
============= Backtest 8040=============
============= Backtest 8042=============
============= Backtest 8043=============
============= Backtest 8044=============
============= Backtest 8047=============
============= Ba

# Summary

In [23]:
results['1101']

[{'EntryDate': Timestamp('2003-01-06 00:00:00'),
  'ExitDate': Timestamp('2003-01-08 00:00:00'),
  'EntryPrice': 12.4,
  'ExitPrice': 13.64,
  'EntryCommission': 20,
  'ExitCommission': 20,
  'EntryTax': 0,
  'ExitTax': 41,
  'TotalCost': 81,
  'HoldingPeriod': 2,
  'Net': 1159,
  'Ret': 0.0935,
  'MaxPriceBetweeenHolding': 14.15,
  'MaxRetBetweeenHolding': 0.1411,
  'MinPriceBetweeenHolding': 12.4,
  'MinRetBetweeenHolding': 0.0},
 {'EntryDate': Timestamp('2003-12-11 00:00:00'),
  'ExitDate': Timestamp('2004-01-05 00:00:00'),
  'EntryPrice': 16.0,
  'ExitPrice': 17.6,
  'EntryCommission': 23,
  'ExitCommission': 25,
  'EntryTax': 0,
  'ExitTax': 53,
  'TotalCost': 101,
  'HoldingPeriod': 25,
  'Net': 1499,
  'Ret': 0.0937,
  'MaxPriceBetweeenHolding': 18.2,
  'MaxRetBetweeenHolding': 0.1375,
  'MinPriceBetweeenHolding': 16.0,
  'MinRetBetweeenHolding': 0.0},
 {'EntryDate': Timestamp('2004-02-10 00:00:00'),
  'ExitDate': Timestamp('2004-03-02 00:00:00'),
  'EntryPrice': 19.7,
  'ExitPr

In [5]:
summary_ = []
bt_day = datetime.today()
bt_day_str = bt_day.strftime("%Y%m%d")
for ticker, result in results.items():
    total_cost = 0
#     total_pnl = 0
    total_net = 0
    trade_num = len(result)
    win_num = 0
    loss_num = 0
    hold_period = []
    rets = []
    for res in result:
        total_net += res['Net']
        total_cost += res['TotalCost']
        win_num += int(res['Net'] > 0)
        loss_num += int(res['Net'] < 0)
        hold_period.append(res['HoldingPeriod'])
        rets.append(res['Ret'])
    summary_.append({
        '代號':ticker,
        '總成本':total_cost,
        '總損益(淨)':total_net,
        '獲利次數':win_num,
        '損失次數':loss_num,
        '總交易次數':trade_num,
        '勝率%':round(win_num / trade_num*100,2) if trade_num else 0,
        '平均報酬':mean(rets),
        '平均持倉時間(日)':mean(hold_period)
    })
    if result:
        DataFrame(result).to_csv(os.path.join(output_path, f'{ticker}_{bt_day_str}.csv'), index=False)
        

In [6]:
sum_df = DataFrame(summary_)#.sort_values(['勝率%','總交易次數'], ascending=False)

sum_df['總損益(淨)'].sum()

525950

In [7]:
sum_df.describe()

,總成本,總損益(淨),獲利次數,損失次數,總交易次數,勝率%,平均報酬,平均持倉時間(日)
count,1795.000000,1795.000000,1795.000000,1795.000000,1795.000000,1795.000000,1728.000000,1728.000000
mean,3505.186630,293.008357,6.362117,5.021170,11.383287,52.852858,0.013247,66.890583
std,8232.668385,30508.732871,3.932586,3.178383,6.274841,21.517154,0.184008,244.690839
min,0.000000,-408126.000000,0.000000,0.000000,0.000000,0.000000,-0.105900,2.000000
25%,1086.500000,-7416.500000,3.000000,3.000000,6.000000,44.095000,-0.014839,19.491667
50%,1851.000000,0.000000,6.000000,5.000000,12.000000,54.550000,0.005183,30.800000
75%,3550.000000,6857.500000,9.000000,7.000000,16.000000,66.670000,0.027300,53.388393
max,255338.000000,282683.000000,18.000000,17.000000,28.000000,100.000000,6.116550,5819.000000


In [11]:
sum_df.to_csv(os.path.join(output_path, 'Summary.csv'),index=False, encoding='utf-8-sig')

In [8]:
sum_df_prob_sorted = sum_df[sum_df['勝率%'] > 70]
sum_df_trade_num_sorted = sum_df[sum_df.總交易次數 >= 10]
sum_df_holding_period_sorted = sum_df[sum_df['平均持倉時間(日)'] <= sum_df['平均持倉時間(日)'].describe()['50%']]

In [9]:
# top_num = 100
prob_Ticker = sum_df_prob_sorted.代號 # .iloc[:top_num]
trade_num_Ticker = sum_df_trade_num_sorted.代號 # .iloc[:top_num]
holding_period_Ticker = sum_df_holding_period_sorted.代號
suitable_Ticker = list(set(prob_Ticker).intersection(trade_num_Ticker).intersection(holding_period_Ticker))

In [10]:
print(sum_df[sum_df.代號.isin(suitable_Ticker)].shape)
sum_df[sum_df.代號.isin(suitable_Ticker)].sort_values("勝率%", ascending=False).head(50)

(74, 9)


,代號,總成本,總損益(淨),獲利次數,損失次數,總交易次數,勝率%,平均報酬,平均持倉時間(日)
1395,5529,1462,21278,10,0,10,100.00,0.093330,21.800000
1538,6488,26502,259848,9,1,10,90.00,0.073790,11.600000
164,1718,1087,10693,12,2,14,85.71,0.064293,18.000000
1216,4303,2740,26980,17,3,20,85.00,0.061100,29.650000
495,2915,5527,54403,14,3,17,82.35,0.058553,26.352941
722,5531,2159,27561,9,2,11,81.82,0.057218,28.636364
174,1732,1269,12321,9,2,11,81.82,0.057336,27.272727
1033,3152,24407,192883,13,3,16,81.25,0.056369,27.500000
1669,8299,34607,282683,17,4,21,80.95,0.055776,26.571429
127,1539,1533,15997,12,3,15,80.00,0.053267,27.266667


In [12]:
sum_df[sum_df.代號.isin(suitable_Ticker)].sort_values("勝率%", ascending=False).to_csv(os.path.join(output_path, 'Summary_prob_holding_tradenum.csv'), index=False)

In [13]:
sum_df[sum_df.代號.isin(suitable_Ticker)].sort_values("勝率%", ascending=False)#.sum()#describe()

,代號,總成本,總損益(淨),獲利次數,損失次數,總交易次數,勝率%,平均報酬,平均持倉時間(日)
1395,5529,1462,21278,10,0,10,100.00,0.093330,21.800000
1538,6488,26502,259848,9,1,10,90.00,0.073790,11.600000
164,1718,1087,10693,12,2,14,85.71,0.064293,18.000000
1216,4303,2740,26980,17,3,20,85.00,0.061100,29.650000
495,2915,5527,54403,14,3,17,82.35,0.058553,26.352941
...,...,...,...,...,...,...,...,...,...
645,4426,4074,4006,10,4,14,71.43,0.036750,14.714286
323,2406,1627,12013,10,4,14,71.43,0.036329,8.214286
1640,8085,1600,8990,12,5,17,70.59,0.033606,10.588235
1179,4107,6078,21032,12,5,17,70.59,0.035147,28.176471
